**<center><font size="40">Advent of Code 2021</font></center>**

# Puzzle 1
## Part I<br>
1. Count the number of times a depth measurement increases from the previous measurement.Input URL: <url>https://adventofcode.com/2021/day/1/input</url><br>
Example:<br>
199 (N/A - no previous measurement)<br>
200 (increased)<br>
208 (increased)<br>
210 (increased)<br>
200 (decreased)<br>
207 (increased)<br>
240 (increased)<br>
269 (increased)<br>
260 (decreased)<br>
263 (increased)<br>
Increased **7** times.<br>

In [1]:
import pandas as pd

#import data from txt file
df_pz1 = pd.read_csv("data/pz01.txt",
                     header=None,
                     names=['depth'])
df_pz1.head(3)

,depth
0,182
1,188
2,204


In [2]:
#find measurements that increase and sum them up
pz1_answer = (df_pz1.depth.diff() > 0).sum()
print("Answer:",pz1_answer)

Answer: 1215


## Part II

Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.<br>
<br>
Instead, consider sums of a **three-measurement sliding window**. Again considering the above example:<br>
<br>
199  A    
200  A B   
208  A B C  <br>
210    B C D<br>
200  E   C D<br>
207  E F   D<br>
240  E F G  <br>
269    F G H<br>
260      G H<br>
263        H<br>
<br>
Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.<br>
<br>
Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.<br>
<br>
In the above example, the sum of each three-measurement window is as follows:
<br>
A: 607 (N/A - no previous sum)<br>
B: 618 (increased)<br>
C: 618 (no change)<br>
D: 617 (decreased)<br>
E: 647 (increased)<br>
F: 716 (increased)<br>
G: 769 (increased)<br>
H: 792 (increased)<br>
<br>
In this example, there are 5 sums that are larger than the previous sum.<br>
<br>
Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [3]:
#create a copy of initial df
df_pz1_2 = df_pz1.copy()

#create column with following three measurement sums
df_pz1_2['3sum'] = df_pz1_2.depth.rolling(3).sum().shift(-2)

df_pz1_2

,depth,3sum
0,182,574.0
1,188,595.0
2,204,612.0
3,203,614.0
4,205,619.0
...,...,...
1995,2573,7715.0
1996,2574,7718.0
1997,2568,7727.0
1998,2576,NaN


In [4]:
#count the number of increases
pz1_2_answer = (df_pz1_2['3sum'].diff() > 0).sum()
print("Part II answer:",pz1_2_answer)

Part II answer: 1150


# Puzzle 2 
## Part I

--- Day 2: Dive! ---
Now, you need to figure out how to pilot this thing.<br>
<br>
It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:<br>
<br>
forward X increases the horizontal position by X units.<br>
down X increases the depth by X units.<br>
up X decreases the depth by X units.<br>
<br>
Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.
<br>
The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:
<br>
forward 5<br>
down 5<br>
forward 8<br>
up 3<br>
down 8<br>
forward 2<br>
<br>
Your horizontal position and depth both start at 0. The steps above would then modify them as follows:<br>
<br>
forward 5 adds 5 to your horizontal position, a total of 5.<br>
down 5 adds 5 to your depth, resulting in a value of 5.<br>
forward 8 adds 8 to your horizontal position, a total of 13.<br>
up 3 decreases your depth by 3, resulting in a value of 2.<br>
down 8 adds 8 to your depth, resulting in a value of 10.<br>
forward 2 adds 2 to your horizontal position, a total of 15.<br>
<br>
After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)<br>
<br>
Calculate the horizontal position and depth you would have after following the planned course. **What do you get if you multiply your final horizontal position by your final depth?**

In [5]:
#read data into df
df_pz2 = pd.read_csv("data/pz02.txt", 
                     header=None, 
                     names=['command'])
df_pz2.head(3)

,command
0,forward 2
1,down 8
2,down 1


In [6]:
#separate directon and the value into separate columns
df_pz2_sep = df_pz2.command.str.rsplit(" ", expand=True).\
    rename(columns={0:"command", 1:"step"})

In [7]:
#change step data type from object to int
df_pz2_sep["step"] = df_pz2_sep.step.astype("int64")
df_pz2_sep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   command  1000 non-null   object
 1   step     1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [8]:
#check for negative values
(df_pz2_sep.step < 0).sum() 

0

In [9]:
#find total forward value
forward1 = df_pz2_sep.step[df_pz2_sep.command == "forward"].sum()
print('Ship went forward:', forward1, "units.")

Ship went forward: 1998 units.


In [10]:
#find total depth increase
depth_inc1 = df_pz2_sep.step[df_pz2_sep.command == "down"].sum()

#find total depth decrease
depth_dec1 = df_pz2_sep.step[df_pz2_sep.command == "up"].sum()

#total depth value
depth_total1 = depth_inc1 - depth_dec1
print("Total depth value:", depth_total1)

Total depth value: 741


In [11]:
#multiply horizontal position to depth value
pz2_answer = forward1 * depth_total1
print("Puzzle 2.1 answer:", pz2_answer)

Puzzle 2.1 answer: 1480518


## Part II
<br>
Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.


In addition to horizontal position and depth, you'll also need to track a third value, **aim**, which also starts at 0. The commands also mean something entirely different than you first thought:<br>

* down X increases your aim by X units.
* up X decreases your aim by X units.
* forward X does two things:
    * It increases your horizontal position by X units.
    * It increases your depth by your aim multiplied by X.
<br>
Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.<br>
<br>
Now, the above example does something different:<br>
<br>
- forward 5 adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
- down 5 adds 5 to your aim, resulting in a value of 5.
- forward 8 adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
- up 3 decreases your aim by 3, resulting in a value of 2.
- down 8 adds 8 to your aim, resulting in a value of 10.
- forward 2 adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.


After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)<br>
<br>
Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. **What do you get if you multiply your final horizontal position by your final depth?**

In [12]:
df_pz2_sep.head()

,command,step
0,forward,2
1,down,8
2,down,1
3,up,7
4,forward,4


In [19]:
#initialize position, aim & depth
pz2_2_pos = 0
pz2_2_aim = 0
pz2_2_depth = 0

#iterate through the data, line-by-line
for i,row in df_pz2_sep.iterrows():
    if row.command == "forward":
        pz2_2_pos += row.step
        pz2_2_depth += pz2_2_aim * row.step
    if row.command == "down":
        pz2_2_aim += row.step
    if row.command == "up":
        pz2_2_aim -= row.step

print("Final position:",pz2_2_pos)
print("Final aim:", pz2_2_aim)
print("Final depth:", pz2_2_depth)
print("Final position x depth:", pz2_2_pos * pz2_2_depth) 

Final position: 1998
Final aim: 741
Final depth: 642047
Final position x depth: 1282809906
